In [35]:
#PROGRAMA MODELO ENTRENADO POR PROMODRONE
#Librerias
import boto3
import io
from PIL import Image, ImageDraw, ExifTags, ImageColor,ImageFont
import json
import pandas as pd

In [20]:
bucket="promodronefaces"
photo="test/c3.jpg"
model="arn:aws:rekognition:us-east-2:864156931594:project/FacesPromoDrone-2/version/FacesPromoDrone-2.2021-05-20T21.40.34/1621564835514"
min_confidence = 70

In [21]:
client=boto3.client('rekognition')

In [11]:
response = client.detect_custom_labels(Image={'S3Object': {'Bucket': bucket, 'Name': photo}}, MinConfidence=min_confidence, ProjectVersionArn=model)

In [29]:
with open('/Users/josemanuelolmedomtz/Desktop/archivoz2.json', 'w') as json_file:
    json.dump(response, json_file)

In [22]:
# Load image from S3 bucket
s3_connection = boto3.resource('s3')
s3_object = s3_connection.Object(bucket,photo)
s3_response = s3_object.get()

stream = io.BytesIO(s3_response['Body'].read())
image=Image.open(stream)

# Ready image to draw bounding boxes on it.
imgWidth, imgHeight = image.size
draw = ImageDraw.Draw(image)

In [32]:
titles = []
lstLabel = []
lstConfidence = []


for face in response['CustomLabels']:
    lstLabel.append(str(face['Name']))
    lstConfidence.append(str(face['Confidence']))

In [37]:
pist = pd.DataFrame(
    {'Label': lstLabel,
     'Confidence': lstConfidence
    })

In [38]:
pist.to_csv("~/Desktop/datacondensedcustom.txt", index=False, sep ='\t')

In [23]:
# calculate and display bounding boxes for each detected custom label
print('Detected custom labels for ' + photo)
for customLabel in response['CustomLabels']:
    print('Label ' + str(customLabel['Name']))
    print('Confidence ' + str(customLabel['Confidence']))
    if 'Geometry' in customLabel:
        box = customLabel['Geometry']['BoundingBox']
        left = imgWidth * box['Left']
        top = imgHeight * box['Top']
        width = imgWidth * box['Width']
        height = imgHeight * box['Height']
        
        fnt = ImageFont.truetype('/Library/Fonts/Arial.ttf', 15)
        draw.text((left,top), customLabel['Name'], fill='#00d400', font=fnt)
        
        print('Left: ' + '{0:.0f}'.format(left))
        print('Top: ' + '{0:.0f}'.format(top))
        print('Label Width: ' + "{0:.0f}".format(width))
        print('Label Height: ' + "{0:.0f}".format(height))
        
        points = ((left,top),(left + width, top),(left + width, top + height),(left , top + height),(left, top))
        draw.line(points, fill='#00d400', width=5)

Detected custom labels for test/c3.jpg
Label black
Confidence 99.88700103759766
Left: 617
Top: 209
Label Width: 114
Label Height: 162
Label latin
Confidence 99.35700225830078
Left: 227
Top: 756
Label Width: 38
Label Height: 43
Label latin
Confidence 96.57099914550781
Left: 311
Top: 759
Label Width: 42
Label Height: 38
Label latin
Confidence 95.32899475097656
Left: 54
Top: 696
Label Width: 42
Label Height: 43
Label latin
Confidence 93.72799682617188
Left: 381
Top: 761
Label Width: 51
Label Height: 38
Label latin
Confidence 92.26899719238281
Left: 133
Top: 718
Label Width: 32
Label Height: 34
Label black
Confidence 83.39700317382812
Left: 896
Top: 437
Label Width: 25
Label Height: 27
Label latin
Confidence 82.48100280761719
Left: 1135
Top: 431
Label Width: 64
Label Height: 156
Label latin
Confidence 80.28500366210938
Left: 93
Top: 703
Label Width: 27
Label Height: 26
Label latin
Confidence 67.31999969482422
Left: 601
Top: 207
Label Width: 138
Label Height: 173
Label latin
Confidence 63.9

In [24]:
image.show()

In [25]:
image.save("aa.jpg")

In [26]:
label_count = len(response['CustomLabels'])
print("Custom labels detected: " + str(label_count))

Custom labels detected: 11
